In [1]:
import pandas as pd
import json

df = pd.read_csv('./custom_data/base_data.csv', encoding='cp949')

### 사업자등록번호를 이용한 산업코드, 주요사업내용 찾기

In [2]:
# 크롤링에 필요한 라이브러리 불러오기
from selenium import webdriver              
from selenium.webdriver import ActionChains  
from selenium.webdriver.common.by import By  
import time
from selenium.common.exceptions import NoAlertPresentException, NoSuchElementException, TimeoutException, ElementNotInteractableException,NoSuchWindowException, NoSuchFrameException, UnexpectedAlertPresentException
import re

# 산업코드가 결측치인 것들 중에서 사업자등록번호를 가져오기
null_list = df.loc[df['산업코드1'].isna()]['사업자등록번호'].unique().tolist()
name_list = df.loc[df['산업코드1'].isna()]['기업명'].unique().tolist()
result_dict = [{'사업자등록번호':[], '기업명':[], '산업코드1':[], '주요사업내용':[], '주소':[]}, {'사업자등록번호':[], '기업명':[]}]

for company_registration_number, corporation_name in zip(null_list, name_list):
    # 01. 비즈인포 검색창 진입 (sleep=2)
    dr = webdriver.Chrome()  
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    dr.set_window_size(1000, 1000)                   
    dr.get('https://www.nicebizinfo.com/cm/CM0100M001GE.nice') 
    time.sleep(2)

    # 02. 검색창에 사업자등록번호를 입력하고 검색버튼 누르기 (sleep=5)
    search_box_biz_info = dr.find_element(By.XPATH, "/html/body/div[2]/div/ul/li[1]/div/form/input")     # 비즈인포 검색창
    search_button_biz_info = dr.find_element(By.XPATH, '/html/body/div[2]/div/ul/li[1]/div/form/button') # 비즈인포 검색 버튼
    time.sleep(2)

    act = ActionChains(dr)
    act.send_keys_to_element(search_box_biz_info, company_registration_number).click(search_button_biz_info).perform() # 사업자등록번호 입력, 버튼 클릭 수행
    time.sleep(5)

    try:
        # 03. 검색결과 클릭하기
        search_result = dr.find_element(By.CSS_SELECTOR, 'body > div.cSection > div > div.cTable.sp3.mb60 > table > tbody > tr.bg > th > span.fz14.fwb.ml10.fErr > a')
        search_result.click()
        time.sleep(5)

        # 03. 검색해서 나온 결과에서 산업코드, 주요사업내용, 주소 가져오기 (sleep=3)
        click_result1 = dr.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/table/tbody/tr[2]/td[2]/div/strong')
        click_result2 = dr.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/table/tbody/tr[1]/td[2]/div/strong')
        time.sleep(2)

        indurstry_code = re.findall('\(([^)]+)', click_result1.text)[0][1:]   # 산업코드 추출
        business_details = re.sub('\(.*\)|\s-\s.*', '',click_result1.text) # 주요사업내용 추출

        result_dict[0]["사업자등록번호"].append(company_registration_number)
        result_dict[0]["기업명"].append(corporation_name) 
        result_dict[0]["산업코드1"].append(indurstry_code)
        result_dict[0]["주요사업내용"].append(business_details)
        result_dict[0]["주소"].append(click_result2.text)
    except NoSuchElementException as e:
        result_dict[1]["사업자등록번호"].append(company_registration_number)
        result_dict[1]["기업명"].append(corporation_name)
        print(f'{company_registration_number} {corporation_name}의 정보가 없습니다.')
    
    except UnexpectedAlertPresentException as e:
        result_dict[1]["사업자등록번호"].append(company_registration_number)
        result_dict[1]["기업명"].append(corporation_name)
        print(f'{company_registration_number} {corporation_name}의 정보가 없습니다.')

1058534234 케이씨크린트래이딩의 정보가 없습니다.
1078550147 블루인더스의 정보가 없습니다.
1208543538 에이비온바이오메드의 정보가 없습니다.
1208547121 엘에프네트웍스송림아울렛점의 정보가 없습니다.
1228540328 대지이앤씨산업(주)인천동부지점의 정보가 없습니다.
1228544834 성원디앤에프부평지점의 정보가 없습니다.
1228549353 대경산업의 정보가 없습니다.
1258537161 코단에코크리트의 정보가 없습니다.
1348548056 세종에스엠에스화산지점의 정보가 없습니다.
1448505224 성지디앤디의 정보가 없습니다.
1808500152 한국교육시스템(주)분당지점코레샵의 정보가 없습니다.
2068515864 세종에스엠에스한국관광용품센타지점의 정보가 없습니다.
2068533241 삼진아트팩지점의 정보가 없습니다.
2158547074 닭익는마을의 정보가 없습니다.
2158547337 대신네트웍스(주)춘천지점의 정보가 없습니다.
2548500899 쎄니트해운대지점의 정보가 없습니다.
2578500637 블레이스홀딩스의 정보가 없습니다.
2598500477 비티캠퍼스의 정보가 없습니다.
2968500078 블루네트웍스중부지사의 정보가 없습니다.
3128563152 신한정밀공업(주)대전공장의 정보가 없습니다.
3148522123 범신사세종지사의 정보가 없습니다.
3148550537 에코로화성지점의 정보가 없습니다.
3198500823 에프티이앤이의 정보가 없습니다.
3538500221 다이캐스탈코리아의 정보가 없습니다.
3808500655 부농 군산지점의 정보가 없습니다.
3928500395 회명애쉬랜드(주)군산의 정보가 없습니다.
4018516930 신창테크전주지점의 정보가 없습니다.
4018518068 아트패브릭의 정보가 없습니다.
4028519532 이앤이솔루션의 정보가 없습니다.
4088528247 예건광주지점의 정보가 없습니다.
4098527412 제희의 정보가 없습니다.
4238500705 엔도비전대구지점의 정보

### 사업자등록번호로 조회되지 않는 기업들은 기업명으로 다시 검색하기

In [3]:
# 검색 결과가 없는 것은 기업명으로 검색해보기
# 크롤링에 필요한 라이브러리 불러오기
from selenium import webdriver              
from selenium.webdriver import ActionChains  
from selenium.webdriver.common.by import By  
import time
from selenium.common.exceptions import NoAlertPresentException, NoSuchElementException, TimeoutException, ElementNotInteractableException,NoSuchWindowException, NoSuchFrameException, UnexpectedAlertPresentException
import re


for company_registration_number, corporation_name in zip(result_dict[1]["사업자등록번호"], result_dict[1]["기업명"]):
    # 01. 비즈인포 검색창 진입 (sleep=2)
    dr = webdriver.Chrome()   
    dr.set_window_size(1000, 1000)                   
    dr.get('https://www.nicebizinfo.com/cm/CM0100M001GE.nice') 
    time.sleep(2)

    # 02. 검색창에 사업자등록번호를 입력하고 검색버튼 누르기 (sleep=5)
    search_box_biz_info = dr.find_element(By.XPATH, "/html/body/div[2]/div/ul/li[1]/div/form/input")     # 비즈인포 검색창
    search_button_biz_info = dr.find_element(By.XPATH, '/html/body/div[2]/div/ul/li[1]/div/form/button') # 비즈인포 검색 버튼
    time.sleep(2)

    act = ActionChains(dr)
    act.send_keys_to_element(search_box_biz_info, corporation_name).click(search_button_biz_info).perform() # 기업명 입력, 버튼 클릭 수행
    time.sleep(5)

    try:
        # 03. 검색결과 클릭하기
        search_result = dr.find_element(By.CSS_SELECTOR, 'body > div.cSection > div > div.cTable.sp3.mb60 > table > tbody > tr.bg > th > span.fz14.fwb.ml10.fErr > a')
        search_result.click()
        time.sleep(5)

        # 03. 검색해서 나온 결과에서 주요사업내용 가져오기 (sleep=3)
        click_result = dr.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/table/tbody/tr[2]/td[2]/div/strong')
        time.sleep(2)

        indurstry_code = re.findall('\(([^)]+)', click_result.text)[0][1:]   # 산업코드 추출
        business_details = re.sub('\(.*\)|\s-\s.*', '',click_result.text) # 주요사업내용 추출

        result_dict[0]["사업자등록번호"].append(company_registration_number)
        result_dict[0]["기업명"].append(corporation_name) 
        result_dict[0]["산업코드1"].append(indurstry_code)
        result_dict[0]["주요사업내용"].append(business_details)
        result_dict[0]["주소"].append(click_result2.text)
        result_dict[1]["사업자등록번호"].remove(company_registration_number)
        result_dict[1]["기업명"].remove(corporation_name)
        
    except NoSuchElementException as e:
        continue
    
    except UnexpectedAlertPresentException as e:
        continue

In [4]:
import json

with open('.custom_data/indurstry_code_missing_value.json','w') as f:
  json.dump(result_dict, f, ensure_ascii=False, indent=4)

- 사업자등록번호, 기업명으로도 검색이 안되는 경우
    - 지사, 지점, 센터 등등 본사에서 파생된 경우는 검색이 안 됐다.
        - 본사의 산업코드, 주요사업내용을 따라가면 될 것 같다.
    - 회사명이 바뀐 경우
        - 에이비온바이오메드의 경우는 에이비온이 바이오메드 사업을 철수하면서 에이비온으로 이름이 변경된 것 같다.
    - 그 외
        - 사업자등록번호가 잘못 등재되어 있는 경우

>찾은 결과

[그냥 안 나오는 기업]
- '블루인더스', => 마스크 파는 곳, 정천식으로 대표자가 바뀜, 사업자등록번호가 다르다
- '한서공조', => 냉난방공조 메이커 회사, 사업자등록번호가 다르다
- '코단에코크리트', => 콘크리트블록 파는 곳, 사업자등록번호가 다르다
- '에프티이앤이', 주식회사 라임의 자회사, 마스크 파는 곳, 실제 대표자명도 다르고 사업자등록번호도 다르다
- '비티캠퍼스', => 차량번호 인식하는 제품 파는 곳, 사업자등록번호가 다르다, 이름도 비티캠퍼스가 아니라 비티컴퍼스임
- '에이비온바이오메드', => 19년 3월 15일 에이비온이 바이오메드사업부(시약 및 의약품 수입, 도소매업)의 영업을 중지함

[지점이라서 안 나오는 기업]
- '엘에프네트웍스송림아울렛점',
- '대지이앤씨산업(주)인천동부지점',
- '성원디앤에프부평지점',
- '세종에스엠에스화산지점',
- '한국교육시스템(주)분당지점코레샵',
- '세종에스엠에스한국관광용품센타지점',
- '삼진아트팩지점',
- '대신네트웍스(주)춘천지점',
- '쎄니트해운대지점',
- '블루네트웍스중부지사',
- '범신사세종지사',
- '에코로화성지점',
- '부농 군산지점',
- '회명애쉬랜드(주)군산',
- '신창테크전주지점',
- '예건광주지점',
- '엔도비전대구지점',
- '성우아이디건설동대구지점',
- '더블유아이케이중부',
- '포스토피아대구센터',
- '네비엔영천사업소',
- '시소아이티부산지사',
- '가양김해지점',
- '포스토피아제주센터',
- '네티브(주)부산지사',
- '정선레미콘남서울공장',
- '코레일관광개발(주)강빛마을펜션'
- '신한정밀공업(주)대전공장'